In [20]:
#MODULE LOADER

try :
    from datetime import date, datetime, timedelta
    print("Datetime imported successfully.")
except Exception as oops : print(oops)
    
import os
import httpx

today = input("What is today's date? (YYYYMMDD): ")
print("+_+_+_+_+_+_+_+_+_+_+_+\n     MODULES READY    \n+_+_+_+_+_+_+_+_+_+_+_+")

#Script built from the provided template https://github.com/akrherz/iem/blob/main/scripts/asos/iem_scraper_example2.py

Datetime imported successfully.
What is today's date? (YYYYMMDD): 20260122
+_+_+_+_+_+_+_+_+_+_+_+
     MODULES READY    
+_+_+_+_+_+_+_+_+_+_+_+


In [21]:
#SUPPORT CELL
profile = input(f"What data profile do you want to pull? Please pay attention to the following options and input your choice exactly: \nPlease note that temperature is in Celsius but all other measurements are imperial.\n    For Temp Only = temp    \n    For RH Only = hurs \n    For Rain Only = precip \n    For Wind Only = sfcw \n    For Temp, RH, Rain, Wind = full \n    For Temp, RH = heat \n    For Temp, Rain = storm \n    For Temp, Wind = mojave \n    For Temp, Wind, Snow = snow \n        ")
yearrun = int(input("What year would you like to start in? (####) "))
monthrun = int(input("What month would you like to start in? (#) "))
dayrun = int(input("What day would you like to start in? (#) "))
startdate = (f"{yearrun}" + f"{monthrun}" + f"{dayrun}")
runpath = int(input("How many months do you want to pull? (#)"))
pulltotal = runpath * 30

        
#        Mick Udby at audby2@huskers.unl.edu

What data profile do you want to pull? Please pay attention to the following options and input your choice exactly: 
Please note that temperature is in Celsius but all other measurements are imperial.
    For Temp Only = temp    
    For RH Only = hurs 
    For Rain Only = precip 
    For Wind Only = sfcw 
    For Temp, RH, Rain, Wind = full 
    For Temp, RH = heat 
    For Temp, Rain = storm 
    For Temp, Wind = mojave 
    For Temp, Wind, Snow = snow 
        snow
What year would you like to start in? (####) 1997
What month would you like to start in? (#) 3
What day would you like to start in? (#) 10
How many months do you want to pull? (#)1


In [22]:
#DATA PULLER
#PULLS MONTHS OF DATA

metarpull = dict(
            temp='data=tmpc',
            hurs='data=relh',
            precip='data=p01i',
            sfcw='data=gust_mph', 
            full='data=tmpc&data=relh&data=relh&data=p01i&data=gust_mph', 
            heat='data=tmpc&data=relh',
            storm='data=tmpc&data=p01i',
            mojave='data=tmpc&data=gust_mph',
            snow='data=tmpc&&data=gust_mph&data=snowdepth',
            )
PULL = metarpull[profile]

print(f"Setting puller for {profile} across {runpath} months starting in {startdate}.")

branchsplit = input(f"For one station to pull, input 'S'\nFor multiple stations to pull, input 'M'\nWhat kind of pull is this?    ").upper()
if branchsplit == 'S' :
    stationID = input(f"What station do you want to pull? Please pay attention to the following options and input your choice exactly: \n    Lincoln = LNK\n    Omaha = OMA\n    Sidney = SNY\n    ONiel = ONL\n    Valentine = VTN\n    Scottsbluff = BFF\n        ").upper()
elif branchsplit == 'M' :
    stateloop = []
    first = input(f"What station do you want first? Please pay attention to the following options and input your choice exactly: \n    Lincoln = LNK\n    Omaha = OMA\n    Sidney = SNY\n    ONiel = ONL\n    Valentine = VTN\n    Scottsbluff = BFF\n    Offut Airforce Base = OFF \n    Imperial = IMP    ").upper()
    stateloop.append(first)
    nxt = input(f"What station do you want to pull next? (These will be separate CSVs)    ").upper()
    stateloop.append(nxt)
    while True :
        last = input(f"\nFor further stations, input the station code.\n\nIf finished compiling the station list, input exactly 'go'    ").upper()
        if last != 'GO' :
            stateloop.append(last)
            print(f"\nCurrent station pull list is {stateloop}.\n")
        elif last == 'GO' :
            print(f"+_+_+_+_+_+_+_+_+_+_+_+\nCurrent station pull list is {stateloop}.\nInitiating pull.\n+_+_+_+_+_+_+_+_+_+_+_+")
            break
        else : 
            print(f"Error arose during station list looping. Contact Mick Udby at audby2@huskers.unl.edu for bug report, but gentle.") 
            break






def fetch(stationID):
    """Download data we are interested in!"""
    localfn = f"{today}_{profile}_{runpath}monthrun{yearrun}_{stationID}.csv"
    if os.path.isfile(localfn):
        print(f"- Cowardly refusing to over-write existing file: {localfn}") #I guess iowans have a sense of humor
        return
    print(f"+ Downloading for {stationID}")
    enddt = date.today() + timedelta(days=pulltotal)
    uri = (
        "http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?"
        f"station={stationID}&{PULL}&year1={yearrun}&month1={monthrun}&day1={dayrun}&"
        f"year2={enddt.year}&month2={enddt.month}&day2={enddt.day}&"
        "tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=empty&trace=0.0001&"
        "direct=yes&report_type=3"
    )
    resp = httpx.get(uri, timeout=300)
    with open(localfn, "w", encoding="utf-8") as fh:
        fh.write(resp.text)

def main():
    """Main loop."""
    fetch(stationID)
    print(f"{stationID} PULL IS FINISHED. PLEASE IMMEDIATELY MOVE YOUR DATA TO THE APPROPRIATE FOLDER TO LEAVE A CLEAN DIRECTORY FOR OTHERS.")

if __name__ == "__main__":
    if branchsplit == 'S' :
        main()
    elif branchsplit == 'M' :
        for stationID in stateloop :
            main()
    else : print("Error arose during final pull looping, when reading branchsplit decision. Contact Mick Udby at audby2@huskers.unl.edu for bug report, but gentle.")

Setting puller for snow across 1 months starting in 1997310.
For one station to pull, input 'S'
For multiple stations to pull, input 'M'
What kind of pull is this?    M
What station do you want first? Please pay attention to the following options and input your choice exactly: 
    Lincoln = LNK
    Omaha = OMA
    Sidney = SNY
    ONiel = ONL
    Valentine = VTN
    Scottsbluff = BFF
    Offut Airforce Base = OFF 
    Imperial = IMP    lnk
What station do you want to pull next? (These will be separate CSVs)    off

For further stations, input the station code.

If finished compiling the station list, input exactly 'go'    vtn

Current station pull list is ['LNK', 'OFF', 'VTN'].


For further stations, input the station code.

If finished compiling the station list, input exactly 'go'    imp

Current station pull list is ['LNK', 'OFF', 'VTN', 'IMP'].


For further stations, input the station code.

If finished compiling the station list, input exactly 'go'    go
+_+_+_+_+_+_+_+_+_+_+_+

In [ ]:
#DATA SAVER